In [1]:
#default_exp callbacks

In [27]:
#export
from fastai.vision import *
from fastai.callbacks import *
from time import time
from fastprogress.fastprogress import format_time

In [28]:
#export
class SaveDistributedModelCallback(TrackerCallback):
    "SaveModelCallback modified for distributed transfer learning - remove torch.load"
    def __init__(self, learn:Learner, monitor:str='val_loss', mode:str='auto', every:str='improvement',
                 name:str='bestmodel', gpu=None):
        super().__init__(learn, monitor=monitor, mode=mode)
        self.every,self.name = every,name
        if self.every not in ['improvement', 'epoch']:
            warn(f'SaveModel every {self.every} is invalid, falling back to "improvement".')
            self.every = 'improvement'
        self.gpu = gpu
      
    def on_train_begin(self, **kwargs:Any)->None:
        "Initializes the best value."
        if not hasattr(self, 'best'):
            print("Initializing self.best")
            self.best = float('inf') if self.operator == np.less else -float('inf')
        
    def jump_to_epoch(self, epoch:int)->None:
        try: 
            self.learn.load(f'{self.name}_{epoch-1}', purge=False)
            print(f"Loaded {self.name}_{epoch-1}")
        except: print(f'Model {self.name}_{epoch-1} not found.')

    def on_epoch_end(self, epoch:int, **kwargs:Any)->None:
        "Compare the value monitored to its best score and maybe save the model."
        if self.every=="epoch": self.learn.save(f'{self.name}_{epoch}')
        else: 
            current = self.get_monitor_value()
            if current is not None and self.operator(current, self.best):
                if not self.gpu: print(f'Better model found at epoch {epoch} with {self.monitor} value: {current}.')
                self.best = current
                self.learn.save(f'{self.name}')

In [29]:
#export
class CSVLogger(LearnerCallback):
    "A `LearnerCallback` that saves history of metrics while training `learn` into CSV `filename`."
    def __init__(self, learn:Learner, filename: str = 'history', append: bool = False): 
        super().__init__(learn)
        self.filename,self.path,self.append = filename,self.learn.path/f'{filename}.csv',append
        self.add_time = True

    def read_logged_file(self):  
        "Read the content of saved file"
        return pd.read_csv(self.path)

    def on_train_begin(self, **kwargs: Any) -> None:
        "Prepare file with metric names."
        self.path.parent.mkdir(parents=True, exist_ok=True)      
        self.file = self.path.open('a') if self.append else self.path.open('w')
        self.file.write(','.join(self.learn.recorder.names[:(None if self.add_time else -1)]) + '\n')
    
    def on_epoch_begin(self, **kwargs:Any)->None:
        if self.add_time: self.start_epoch = time()
        
    def on_epoch_end(self, epoch: int, smooth_loss: Tensor, last_metrics: MetricsList, **kwargs: Any) -> bool:
        "Add a line with `epoch` number, `smooth_loss` and `last_metrics`."
        last_metrics = ifnone(last_metrics, [])
        stats = [str(stat) if isinstance(stat, int) else '#na#' if stat is None else f'{stat:.6f}'
                 for name, stat in zip(self.learn.recorder.names, [epoch, smooth_loss] + last_metrics)]
        if self.add_time: stats.append(format_time(time() - self.start_epoch))
        str_stats = ','.join(stats)
        self.file.write(str_stats + '\n')
        self.file.flush()
        os.fsync(self.file.fileno())

    def on_train_end(self, **kwargs: Any) -> None:  
        "Close the file."
        self.file.close()

In [30]:
from local.segmentation.dataset import SemanticSegmentationData
from local.segmentation.metrics import *
# test data creation
PATH = Path("/home/turgutluk/.fastai/data/camvid")
IMAGES = "images"
MASKS = "labels"
CODES = "codes.txt"
TRAIN, VALID, TEST = "train.txt", "valid.txt", "test.txt"
ssdata = SemanticSegmentationData(PATH, IMAGES, MASKS, CODES, TRAIN, VALID, TEST, sample_size=None, bs=4, size=112)
data = ssdata.get_data()

In [31]:
learn = unet_learner(data, models.resnet34); learn.metrics = [partial(foreground_acc, void_code=30)]

In [32]:
#export
def _get_metric_name(f):
    try: return f.func.__name__
    except: return f.__name__

In [33]:
save_cb = SaveDistributedModelCallback(learn, monitor=_get_metric_name(learn.metrics[0]))
csvlog_cb = CSVLogger(learn, './history.csv', append=True)
cbs = [save_cb, csvlog_cb]

In [ ]:
learn.fit_one_cycle(1, callbacks=cbs)

Initializing self.best


epoch,train_loss,valid_loss,foreground_acc,time


In [ ]:
save_cb.best

In [ ]:
learn.fit_one_cycle(1, callbacks=cbs)

In [ ]:
from local.notebook.export import notebook2script
notebook2script("03_distributed.ipynb")